In [1]:
import csv
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly
import os
import tqdm
import json
from collections import defaultdict
import json_lines
import datetime
import matplotlib.pyplot as plt
from statistics import median
from ipywidgets import interact

# 파일 불러오기

In [2]:
# 성지 후보군 기사들
with open('../pilgrim data/result/pilgrim_candidates.json') as f:
    cands = json.load(f)

# sid 별 매칭 정보가 담긴 파일
with open('../pilgrim data/sid_to_cat.csv', 'r', encoding = 'utf-8') as g:
    reader = csv.reader(g)
    sid_to_cat = {line[0]: line[1] for line in reader}
    del sid_to_cat['\ufeffsid']

In [42]:
i = 1
for cand in [a for a in cands]:
    cand = cand[1]
    print('{}. {}'.format(i, cand['title']))
    print('URL: {}'.format(cand['url']))
    scores.append(score)
    i += 1

1. 중증장애인 부부의 '아름다운 기부'
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0005964326
2. 부안여고 선플누리단 'YES 2011 사진 전시회' 21일까지 의회 전시
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0004130332
3. '궁금한 이야기Y' 학교 폭력 피해자를 성추행 가해자로 바꾼 쪽지 한 장
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=213&aid=0000523205
4. [단독] 최태원 내연녀 김 씨, 어머니 대신 금감원 출석시켜
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=004&oid=056&aid=0010276883
5. 安 "청년, 투표 안하니 지원법 저조…어느 당 찍든 투표해야"
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0007125210
6. '데뷔' 방탄소년단 "롤모델은 빅뱅, 살아남겠다"
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=108&aid=0002225969
7. 국산 경유차, 연비 경쟁에 '폐질환 유발' 오염 물질 외면
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=103&oid=437&aid=0000055288
8. 뇌성마비 부부 애타는 사연 “눈으로 안을 수 밖에 없는 딸…”
URL: https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=

# Clustering 기준 만들기

## timeseries에서 댓글 비중이 가장 크게 증가한 구간의 증가율

In [64]:
def max_increase (timeline):
    maximum = 0
    cnts = np.cumsum(list(timeline.values())) / sum(list(timeline.values()))
    components = list(zip(list(timeline.keys()), cnts))
    for idx in range(1, len(components)):
        former_time, former_rate = components[idx-1]
        now_time, now_rate = components[idx]
        dx = int((datetime.datetime.strptime(now_time, '%Y%m%d') - datetime.datetime.strptime(former_time, '%Y%m%d')).days)
        dy = now_rate - former_rate
        rate = dy
        if rate > maximum:
            maximum = rate
    
    return maximum

## 전체 time 중 초반 20% 구간에서의 댓글이 차지하는 비율

In [65]:
def initial_rush (info):
    # 초반 20프로 구간에서의 댓글 증가량 확인.
    timeline = info['timeline']
    cnts = np.cumsum(list(timeline.values())) / sum(list(timeline.values()))
    published_date = datetime.datetime.strptime(info['timestamp'][:10], '%Y-%m-%d')
    total_length = (datetime.datetime(2019,12,31) - published_date).days
    target_length = int(total_length * .2)
    dates = list(map(lambda x: datetime.datetime.strptime(x, '%Y%m%d'), list(timeline.keys())))
    for i in range(len(dates)):
        day = dates[i]
        if (day - published_date).days > target_length:
            return cnts[i]
        
    return cnts[i]

In [87]:
data = []
for (aid,cand) in cands:
    data.append((initial_rush(cand), max_increase(cand['timeline'])))
data

- 대략적으로 생각해보았을 때 각 값을 크고, 작게 나누는 총 4개의 경우의 수로 분류할 수 있을 것임.
- 이를 기준으로 k = 4로 KMeans clustering을 진행함.

In [102]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters= 4, random_state=0).fit(data)
result = kmeans.labels_
print(kmeans.cluster_centers_)

[[0.24997858 0.12573771]
 [0.82609605 0.12955258]
 [0.13515566 0.53281044]
 [0.87753631 0.58944667]]


# 분석 결과
- cluster 1: 초반에 적은 비중의 댓글을 가지나 완만하게 계속 증가함.
- cluster 2: 초반에 비교적 많은 댓글이 달리고 이후 완만하게 증가.
- cluster 3: 초반에 적은 비중의 댓글을 가지나 폭발적인 증가 구간이 존재.
- cluster 4: 초반에 비교적 많은 댓글이 달리고 이후 폭발적인 증가 구간이 존재.

In [105]:
clusters = defaultdict(list)
idx = 0
for (aid, cand) in cands:
    label = result[idx]
    clusters[label].append(cand)
    idx += 1

## 각 clsuter 가 얼마나 많은 '성지순례' 언급 성지를 가지고 있는 지 확인.

In [110]:
for c in list(clusters.keys()):
    cluster = clusters[c]
    total = len(cluster)
    real_pilg = 0
    for article in cluster:
        if article['is_pilgrim'] == 'YES':
            real_pilg += 1
    print('{}: {:.3f} / size: {}'.format(c+1, real_pilg / total, len(cluster)))
    
real_pilg = 0
for (aid, cand) in cands:
    if cand['is_pilgrim'] == 'YES':
        real_pilg += 1
print('TOTAL: {:.3f}'.format(real_pilg / len(cands)))

1: 0.297 / size: 222
2: 0.177 / size: 328
3: 0.281 / size: 114
4: 0.146 / size: 123
TOTAL: 0.221


--> Cluster 1과 3이 평균보다 많은 '성지순례' 언급 성지를 가지고 있으며, 2와 4는 그와 반대임.

## 각 clsuter 별로 pilgrim score의 평균 계산.

In [115]:
def total_score(info):
    '''
    input: 각 기사에 대한 정보 (timeline, timestamp, etc)
    output: 각 기사 별 pilgrim score

    OLD_SCORE: 각 댓글이 기사 생성일로부터 오래 될수록 가중치를 부여하여 매기는 점수
    RECENT_SCORE: 최근에 작성된 댓글로 이루어진 성지일수록 높은 점수를 받도록 구성.
                  같은 달에 쓰여진 댓글의 경우 같은 가중치를 갖도록. (12월 30일과 12월 1일은 거시적인 관점에서 큰 차이 없게.)
    CONCENTRATED_SCORE: 특정 기간에 집중되어 만들어진 성지를 찾고자 함.
                        이게 너무 높으면 좋은 성지라고 보기는 어렵도록 설정.
                        이 값이 클 때에만 전체 스코어에 큰 영향을 끼치도록, 작을 수록 별 차이 없도록 최종 식에서 설정.
    '''
    N = 365
    published_date = datetime.datetime.strptime(
        info['timestamp'][:10], '%Y-%m-%d')
    timeline = info['timeline']
    dates = list(map(lambda x: (x, datetime.datetime.strptime(
        x, '%Y%m%d')), list(timeline.keys())))
    length = (dates[-1][1] - published_date).days

    old_score = 0
    recent_score = 0
    days = []
    pilg_count = 0

    for string, date in dates:
        diff = (date - published_date).days
        if diff >= N:
            old_score += diff * timeline[string]
            recent_score += 1 / \
                ((datetime.datetime(2019, 12, 31) -
                  date).days // 30) * timeline[string]
            pilg_count += timeline[string]
            for i in range(timeline[string]):
                days.append(diff)

    os = old_score / pilg_count / 365
    rs = recent_score / pilg_count * 100  # 절대적인 크기를 맞추어주기 위해서
    amount = len(days)
    # 집중도를 계산하는 데에 존재하는 아웃라이어들을 제거.
    days = days[5:-
                5] if amount > 50 else days[int(amount * .1): int(amount * .9)]
    cs = 10e-3 if np.std(days) == 0 else np.std(days)
    cs = 5 if 1/cs > 5 else 1/cs
    cs = int(cs+1)
--> Cluster 1과 3이 평균보다 많은 '성지순례' 언급 성지를 가지고 있으며, 2와 4는 그와 반대임.
    return os, rs, cs, (os * rs) / cs * pilg_count

for c in list(clusters.keys()):
    cluster = clusters[c]
    scores = []
    for article in cluster:
        scores.append(total_score(article)[-1])
    print('{}: {:.3f} / size: {}'.format(c+1, np.mean(scores), len(cluster)))
    
scores = []
for (aid, cand) in cands:
    scores.append(total_score(cand)[-1])
print('TOTAL: {:.3f}'.format(np.mean(scores)))

1: 1184.740 / size: 222
2: 305.072 / size: 328
3: 1496.269 / size: 114
4: 325.580 / size: 123
TOTAL: 728.967


--> Cluster 1과 3이 비교적 상당히 높은 pilgrim score를 보임.

# 클러스터 별 그래프 확인

In [122]:
@interact (idx = (0,len(clusters[2])))
def show_cdf (idx = 0):
    fig = go.Figure()
    target = clusters[2][idx]
    
    #기사정보 표시하기
    print('Title: {}'.format(target['title']))
    print('Published: {}'.format(target['timestamp']))
    print('URL: {}'.format(target['url']))
    print('직접적인 성지 언급: {} \n'.format(target['is_pilgrim']))
    
    timeline = target['timeline']
#     for k,v in timeline.items():
#         print('{} : {} comments'.format(k,v))
    
    published_date = datetime.datetime.strptime(target['timestamp'][:10], '%Y-%m-%d')
    dates = list(map(lambda x: datetime.datetime.strptime(x, '%Y%m%d'), list(target['timeline'].keys())))
    cnts = np.cumsum(list(target['timeline'].values())) / sum(list(target['timeline'].values()))
    dates.insert(0, published_date)
    cnts = np.insert(cnts, 0, 0)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = dates, y = cnts, mode='markers+lines'))
    fig.update_yaxes(range = [0,1])
    # fig.update_xaxes(range=[published_date,
    #                                datetime.datetime(2019, 12, 31)])
    fig.update_xaxes(range=[published_date,
                                   datetime.datetime(2019, 12, 31)])
    fig.show()

interactive(children=(IntSlider(value=0, description='idx', max=114), Output()), _dom_classes=('widget-interac…